In [5]:
!pip install langchain faiss-cpu sentence-transformers gradio huggingface_hub


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install -U email-validator

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
pip install gradio langchain faiss-cpu sentence-transformers huggingface_hub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
pip install gradio langchain faiss-cpu sentence-transformers transformers

Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain_huggingface import HuggingFacePipeline
from langchain.docstore.document import Document
from langchain.llms import HuggingFaceHub
import os
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

In [3]:
def load_text_file(file_path="C:\\Users\\Akshata Pandit\\OneDrive\\Documents\\knowledge_base.txt"):
    with open(file_path,"r",encoding="utf-8") as f:
        text=f.read()
    splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=50)
    chunks=splitter.split_text(text)
    print(f"Loaded and split text into {len(chunks)} chunks.")
    return [Document(page_content=chunk) for chunk in chunks]

In [4]:
def create_vector_store(documents):
    embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    print("Creating FAISS vector store...")
    vectorstore=FAISS.from_documents(documents, embeddings)
    print("Vector store created.")
    return vectorstore

In [5]:
def create_qa_chain():
    model_name="google/flan-t5-base"
    print(f"Loading tokenizer and model for {model_name}...")
    tokenizer=AutoTokenizer.from_pretrained(model_name)
    model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
    pipe=pipeline(
        "text2text-generation", 
        model=model, 
        tokenizer=tokenizer, 
        max_length=256,
        device=-1  
    )
    
    local_llm=HuggingFacePipeline(pipeline=pipe)
    
    print("Loading QA chain...")
    qa_chain = load_qa_chain(local_llm, chain_type="stuff")
    print("QA chain ready.")
    return qa_chain

In [6]:
documents=load_text_file()
vectorstore=create_vector_store(documents)
qa_chain=create_qa_chain()

Loaded and split text into 20 chunks.


C:\Users\Akshata Pandit\AppData\Local\Temp\ipykernel_2944\3194352264.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


Creating FAISS vector store...
Vector store created.
Loading tokenizer and model for google/flan-t5-base...


Device set to use cpu


Loading QA chain...
QA chain ready.


C:\Users\Akshata Pandit\AppData\Local\Temp\ipykernel_2944\1111294708.py:17: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(local_llm, chain_type="stuff")


In [7]:
def chatbot(query):
    try:
        docs=vectorstore.similarity_search(query)
        print(f"Retrieved {len(docs)} documents for query: {query}")
        answer=qa_chain.run(input_documents=docs, question=query)
        print(f"Answer: {answer}")
        return answer
    except Exception as e:
        print(f"Error during QA: {e}")
        return f"⚠️ Error: {str(e)}"

In [8]:
iface=gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask your question..."),
    outputs="text",
    title="Conversational E Commerece AI Bot",
    description="Ask questions based on your custom knowledge base."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [10]:
iface=gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask your question..."),
    outputs="text",
    title="Conversational E Commerece AI Bot",
    description="Ask questions based on your custom knowledge base."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Retrieved 4 documents for query: is refund available for my order?
Answer: Yes, refunds are available for orders placed after the first day of checkout.


In [11]:
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask your question..."),
    outputs="text",
    title="Conversational E Commerece AI Bot",
    description="Ask questions based on your custom knowledge base."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Retrieved 4 documents for query: what is return policy?
Answer: You can return any item within 30 days of purchase with a valid receipt. The product must be unused and in its original packaging.


In [12]:
iface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.


In [13]:
iface.launch(True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
* To create a public link, set `share=True` in `launch()`.
